In [1]:
# import pandas as pd
# import numpy as np

# # Number of rows
# n = 5000

# # Generating order IDs (unique)
# order_ids = np.arange(1, n+1)

# # Generating customer IDs (random)
# customer_ids = np.random.randint(1000, 9999, n)

# # Cities and states from the American Southeast
# cities = ["Atlanta", "Miami", "Charlotte", "Nashville", "Orlando"]
# states = ["GA", "FL", "NC", "TN", "FL"]

# city_state = list(zip(cities, states))
# chosen_city_state = np.random.choice(len(city_state), n)
# cities_column = [city_state[i][0] for i in chosen_city_state]
# states_column = [city_state[i][1] for i in chosen_city_state]

# # Generating random order dates
# date_range = pd.date_range(start="2020-01-01", end="2023-12-31")
# order_dates = np.random.choice(date_range, n)

# # Total units and revenue
# total_units = np.random.randint(1, 6, n)
# revenue = total_units * np.random.randint(50, 500, n)

# # Furniture items, categories, brands
# products = ["Sofa", "Coffee Table", "Dining Table", "Chair", "Bookshelf"]
# categories = ["Living Room", "Living Room", "Dining Room", "Office", "Living Room"]
# brands = ["Homely", "CraftsWell", "Elegance", "ComfortHome", "UrbanFurnish"]

# product_category = list(zip(products, categories))
# chosen_product_category = np.random.choice(len(product_category), n)
# product_column = [product_category[i][0] for i in chosen_product_category]
# category_column = [product_category[i][1] for i in chosen_product_category]
# brand_column = np.random.choice(brands, n)

# # Store name
# store_name = ["Peachtree Home Interiors"] * n

# # Creating the dataframe
# df = pd.DataFrame({
#     'order_id': order_ids,
#     'customer_id': customer_ids,
#     'city': cities_column,
#     'state': states_column,
#     'order_date': order_dates,
#     'total_units': total_units,
#     'revenue': revenue,
#     'product_name': product_column,
#     'category_name': category_column,
#     'brand_name': brand_column,
#     'store_name': store_name
# })

# # Saving to csv
# csv_filename = df.to_csv('furniture_store_data.csv', index=False)

In [2]:
pip install dash jupyter-dash

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import plotly.express as px
from dash import Dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go

In [4]:
app = Dash(__name__)

df = pd.read_csv('furniture_store_data.csv')

city_coords = {
    'Atlanta': [33.7490, -84.3880],
    'Miami': [25.7617, -80.1918],
    'Charlotte': [35.2271, -80.8431],
    'Nashville': [36.1627, -86.7816],
    'Orlando': [28.5383, -81.3792]
}

revenue_by_city = df.groupby('city').sum().reset_index()  


# Normalize function to map revenue to bubble size
def normalize_size(value, min_val, max_val, size_min, size_max):
    normalized = (value - min_val) / (max_val - min_val)  
    return normalized * (size_max - size_min) + size_min

min_revenue, max_revenue = revenue_by_city['revenue'].min(), revenue_by_city['revenue'].max()
city_list = revenue_by_city['city'].unique().tolist()
df['order_date'] = pd.to_datetime(df['order_date'])

# # Revenue per Year
# def revenue_per_year():
#     yearly_revenue = df.groupby(df['order_date'].dt.year)['revenue'].sum(numeric_only=True).reset_index()
#     fig = px.bar(yearly_revenue, x='order_date', y='revenue', title='Revenue per Year',
#                  color='order_date', color_discrete_sequence=px.colors.qualitative.Pastel1)
#     fig.update_layout(showlegend=False) 
#     return fig

def revenue_per_year():
    yearly_revenue = df.groupby(df['order_date'].dt.year)['revenue'].sum(numeric_only=True).reset_index()
    colors = px.colors.qualitative.Pastel1[:len(yearly_revenue)]
    fig = px.bar(yearly_revenue, x='order_date', y='revenue', title='Revenue per Year')
    fig.update_traces(marker_color=colors)  # Set bar colors
    #fig.update_layout(showlegend=False)  # Remove legend
    fig.update_xaxes(tickvals=yearly_revenue['order_date'].tolist(), ticktext=yearly_revenue['order_date'].astype(str).tolist())  # Fix x-axis tick labels
    return fig


def sold_per_product():
    product_volume = df.groupby('product_name').agg(total_units=('total_units', 'sum')).reset_index()
    fig = px.bar(product_volume, x='product_name', y='total_units', title='Units Sold per Product',
                 color='product_name', color_discrete_sequence=px.colors.qualitative.Pastel1)
    fig.update_yaxes(range=[2500, 3200])
    return fig

def revenue_per_month():
    df['year'] = df['order_date'].dt.year
    df['month'] = df['order_date'].dt.month
    monthly_revenue = df.groupby(['year', 'month'])['revenue'].sum(numeric_only=True).reset_index()
    fig = px.line(monthly_revenue, x='month', y='revenue', color='year', title='Monthly Revenue per Year',
                  color_discrete_sequence=px.colors.qualitative.Pastel1)
    return fig

def category_pie():
    brand_volume = df.groupby('brand_name')['total_units'].sum().reset_index()
    fig = px.pie(brand_volume, names='brand_name', values='total_units', title='Distribution by Brand',
                 color_discrete_sequence=px.colors.qualitative.Pastel1)
    return fig


app.layout = html.Div([
    html.H1("Furniture Store Revenue Analysis"),
    html.Div([
        dcc.Dropdown(id='city-selector', multi=True, value=['All'], options=[{'label': city, 'value': city} for city in city_list]),
        dcc.RadioItems(id='annotation-toggle', options=[{'label': 'Show City Labels', 'value': 'show'}, {'label': 'Hide City Labels', 'value': 'hide'}], value='show'),
        dcc.Graph(id='us-map', style={'height': '30%'})
    ]),
    html.Div([dcc.Graph(id='yearly-revenue', figure=revenue_per_year(), style={'height': '17.5%'}),]),
    html.Div([dcc.Graph(id='product-volume', figure=sold_per_product(), style={'height': '17.5%'}),]),
    html.Div([dcc.Graph(id='category-pie', figure=category_pie(), style={'height': '17.5%'}),]),
    html.Div([dcc.Graph(id='monthly-revenue', figure=revenue_per_month(), style={'height': '17.5%'}),]),
])

# Callback function
@app.callback(
    Output('us-map', 'figure'),
    Output('city-selector', 'value'),
    Input('city-selector', 'value'),
    Input('annotation-toggle', 'value')
)
    
def update_map(selected_values, annotation_status):
    if 'All' in selected_values:
        filtered_df = revenue_by_city
        selected_values = city_list + ['All']
    else:
        filtered_df = revenue_by_city[revenue_by_city['city'].isin(selected_values)]

    mode = 'markers+text' if annotation_status == 'show' else 'markers'
    
    fig = go.Figure(data=go.Scattergeo(
        lon=[city_coords[city][1] for city in filtered_df['city']],
        lat=[city_coords[city][0] for city in filtered_df['city']],
        mode=mode,
        marker=dict(
            size=[normalize_size(rev, min_revenue, max_revenue, 10, 30) for rev in filtered_df['revenue']],
            opacity=0.8,
            autocolorscale=True,
            symbol='circle',
            colorscale='Blues',
            cmin=min_revenue,
            color=filtered_df['revenue'],
            cmax=max_revenue,
            colorbar_title="Revenue $"
        ),
        text=filtered_df['city'] + ': $' + filtered_df['revenue'].astype(str),
        textposition='top center',
        hoverinfo="skip"  
    ))

    fig.update_layout(
        title='Revenue by City Store',
        geo=dict(
            scope='usa',
            showland=True
        )
    )
    return fig, selected_values

if __name__ == '__main__':
    app.run_server(debug=True)
